In [1]:
import pandas as pd
import geopandas as gpd
import osmnx  as ox
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon, GeometryCollection
import numpy as np
import random
import math
from math import sin, cos, sqrt, atan2, radians
import datetime
from datetime import timedelta
import pyproj
from pyproj import Transformer
from pyproj import CRS
import folium
from folium import Map
from folium.plugins import HeatMap
from folium import plugins
from folium.plugins import HeatMapWithTime
import warnings
warnings.filterwarnings("ignore")

In [2]:
place_name = 'Tartu, Tartu linn, Tartu maakond, Estonia'

In [3]:
%%time
graph = ox.graph_from_place(place_name, network_type='walk')

CPU times: total: 23.3 s
Wall time: 25 s


In [4]:
%%time
graph_proj = ox.project_graph(graph)

CPU times: total: 14.4 s
Wall time: 15.3 s


In [5]:
%%time
nodes = ox.graph_to_gdfs(graph_proj, nodes=True, edges=False)

CPU times: total: 797 ms
Wall time: 823 ms


In [6]:
graph_crs=nodes.crs

In [7]:
crs_utm = graph_crs
crs_4326 = CRS.from_epsg(4326)
transformer_to_WGS= Transformer.from_crs(crs_utm, crs_4326, always_xy=True)
# transformer_to_UTM= Transformer.from_crs(crs_4326, crs_utm, always_xy=True)

In [8]:
gdf_buildings = ox.geometries_from_place(place_name, tags={'building':True})

In [9]:
gdf_amenities=ox.geometries_from_place(place_name, tags={"amenity":True, "leisure":True, 'tourism':True})

C:\Users\79219\anaconda3\envs\environment\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\79219\anaconda3\envs\environment\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [10]:
gdf_hw=gdf_buildings.loc[:, ['geometry', 'name']].reset_index()
gdf_hw=gdf_hw.to_crs(graph_crs)

In [11]:
gdf_hw['center_x']=gdf_hw['geometry'].centroid.x
gdf_hw['center_y']=gdf_hw['geometry'].centroid.y

In [12]:
gdf_hw['nearest_node_id']=ox.distance.nearest_nodes(graph_proj, gdf_hw['center_x'], gdf_hw['center_y'])

In [13]:
gdf_hw['node_y']=list(nodes.loc[gdf_hw['nearest_node_id']]['y'])
gdf_hw['node_x']=list(nodes.loc[gdf_hw['nearest_node_id']]['x'])

In [14]:
gdf_hw['distance_to_node']=ox.distance.euclidean_dist_vec\
(gdf_hw['node_y'],gdf_hw['node_x'], gdf_hw['center_y'], gdf_hw['center_x'] )

In [15]:
gdf_event=gdf_amenities.loc[:, ['geometry', 'name']].reset_index()
gdf_event=gdf_event.to_crs(graph_crs)

In [16]:
gdf_event['center_x']=gdf_event['geometry'].centroid.x
gdf_event['center_y']=gdf_event['geometry'].centroid.y

In [17]:
gdf_event['nearest_node_id']=ox.distance.nearest_nodes(graph_proj, gdf_event['center_x'], gdf_event['center_y'])

In [18]:
gdf_event['node_y']=list(nodes.loc[gdf_event['nearest_node_id']]['y'])
gdf_event['node_x']=list(nodes.loc[gdf_event['nearest_node_id']]['x'])

In [19]:
gdf_event['distance_to_node']=ox.distance.euclidean_dist_vec\
(gdf_event['node_y'],gdf_event['node_x'], gdf_event['center_y'], gdf_event['center_x'] )

In [20]:
def get_meaningful_locations(G, gdf_hw, gdf_event): 
    while True: 
        home_id=random.randint(0, len(gdf_hw)-1)
        work_id=random.randint(0, len(gdf_hw)-1)
        home_node=nodes.loc[gdf_hw.iloc[home_id]['nearest_node_id']]
        work_node=nodes.loc[gdf_hw.iloc[work_id]['nearest_node_id']]
        distance_to_h=gdf_hw.iloc[home_id]['distance_to_node']
        distance_to_w=gdf_hw.iloc[work_id]['distance_to_node']
        final_distance = ox.distance.euclidean_dist_vec(home_node['y'], home_node['x'], work_node['y'], work_node['x'])\
                    + distance_to_h+distance_to_w
        if final_distance >=300: # meters
            regular_locations_ids=[]
            number_of_regular_locations=random.randint(3,5)
            i=0
            while i <=number_of_regular_locations: 
                regular_id=random.randint(0, len(gdf_event)-1)
                regular_node=nodes.loc[gdf_event.iloc[regular_id]['nearest_node_id']]
                if ox.distance.great_circle_vec(regular_node['y'], regular_node['x'], work_node['y'], work_node['x'], earth_radius=6371009)>=200: # meters
                    regular_locations_ids.append(regular_id)
                    i+=1                    
                else: 
                    continue
            return home_id, work_id, regular_locations_ids
        else: 
            continue

In [21]:
n=-1
def get_regular_or_random_loc(regular_location_ids, gdf_event):
    global n
    while True: 
        random_id=[random.randint(0, len(gdf_event)-1)]
        array_of_ids=np.array([regular_location_ids, random_id], dtype='object')
        choose_reg_or_random=np.random.choice(array_of_ids,1,p=[0.6,0.4])[0]
        event_id=np.random.choice(choose_reg_or_random)
        if event_id!=n:
            event_x=gdf_event.iloc[event_id]['center_x']
            event_y=gdf_event.iloc[event_id]['center_y']
            event_node_id=gdf_event.iloc[event_id]['nearest_node_id']
            n=event_id
            return event_node_id, event_x, event_y, event_id
        else: 
            continue

In [22]:
def get_static_points(startlon, startlat, user, data_array, time_start, time_end):
    time_start+=timedelta(minutes=1)
    startlon, startlat=transformer_to_WGS.transform(startlon, startlat)
    while time_start<time_end:
        random_minutes=random.randint(1,5)
        possible_forward_azimuth=random.randint(0,360)
        possible_distance=random.randint(0,5) #metres
        endLon,endLat,backAzimuth = (pyproj.Geod(ellps='WGS84')
            .fwd(startlon,startlat,possible_forward_azimuth,possible_distance))
        time_gps=time_start
        data_array.append([user+1, time_gps, endLon, endLat])
        time_start+=timedelta(minutes=random_minutes)
    return time_start.round(freq='S')

In [23]:
def get_points_on_path(path, number_of_points): 
    distances = np.linspace(0, path.length, number_of_points)
    points = [path.interpolate(distance) for distance in distances]
    return points

In [24]:
def get_chaotic_point(point_start, point_end, radius_of_buffer):
    points_intersection=point_start.buffer(radius_of_buffer).intersection(point_end.buffer(radius_of_buffer))
    path_between_points=LineString([point_start, point_end])
    final_intersection=points_intersection.intersection(path_between_points.buffer(2))
    min_x, min_y, max_x, max_y = final_intersection.bounds
    while True: 
        chaotic_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (chaotic_point.within(final_intersection)):
            return chaotic_point
        else: 
            continue

In [25]:
def get_moving_points(G, start_node, end_node,  start_coords, end_coords, user, data_array, time_start,\
                      length_of_distance_m=10, mean_velocity_ms=1.1): 
    
    route = ox.distance.shortest_path(G, start_node, end_node, weight="length")
    route_nodes = nodes.loc[route]
    route_list=list(route_nodes.geometry.values)
    route_list.insert(0,Point(start_coords[0], start_coords[1])) 
    route_list.append(Point(end_coords[0], end_coords[1]))
    path = LineString(route_list)
    
    if path.length<=length_of_distance_m: 
        number_of_points=2
    else: 
        number_of_points=math.ceil(path.length/length_of_distance_m)
    
    points=get_points_on_path(path, number_of_points)

    for i in range(number_of_points):
        endLon, endLat = transformer_to_WGS.transform(points[i].x, points[i].y)
        if i != number_of_points-1: 
            chaotic_point=get_chaotic_point(points[i], points[i+1], radius_of_buffer=length_of_distance_m)
            distance_to_chaotic_point=LineString([points[i],chaotic_point]).length
            if distance_to_chaotic_point < 2.2: 
                time_to_chaotic_point=2
            else: 
                time_to_chaotic_point=distance_to_chaotic_point/mean_velocity_ms
            time_gps=time_start.round(freq='S')
            time_start+=timedelta(seconds=time_to_chaotic_point)
            data_array.append([user+1, time_gps, endLon, endLat])
            
            endLon, endLat = transformer_to_WGS.transform(chaotic_point.x, chaotic_point.y)
            distance_to_next_point=LineString([chaotic_point, points[i+1]]).length
            if distance_to_next_point < 2.2: 
                time_to_next_point=2
            else: 
                time_to_next_point=distance_to_next_point/mean_velocity_ms
            time_gps=time_start.round(freq='S')
            time_start+=timedelta(seconds=time_to_next_point) 
            data_array.append([user+1, time_gps, endLon, endLat])

        else:
            endLon, endLat = transformer_to_WGS.transform(points[i].x, points[i].y)
            time_gps=time_start.round(freq='S')
            data_array.append([user+1, time_gps, endLon, endLat])
    return time_start.round(freq='S')

In [26]:

def random_plot(day_string, list_of_locations, day_of_week, number_of_events): 
    list_of_events_id=[]
    for r in range (number_of_events):
        result=get_regular_or_random_loc(user_locations[user][2], gdf_event)
        event_list=[result[0], result[1], result[2]]
        list_of_events_id.append(result[3])
        list_of_locations.append(event_list)
    
    with open(FILE_NAME, 'a') as file: 
        file.write(f"Day of week: {day_of_week}\n")
        file.write(f'Number of events: {number_of_events}\n')
        file.write(f"Event's ids: {list_of_events_id}\n")
        file.write(f'Length of list locations: {len(list_of_locations)}\n')        
    
    i=0
    
    
    if day_of_week>=6 and len(list_of_locations)>1: 
        stay_activity_time=get_static_points(list_of_locations[i][1], list_of_locations[i][2], user, gps_data_array, \
                                             time_start=another_day_start, time_end=day+timedelta(hours=random.randint(10,14)))
        with open(FILE_NAME, "a") as file: 
            file.write(f'It is weekend and the user stayed at home from {another_day_start} to {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
            
    elif day_of_week<6:    
        
        stay_activity_time=get_static_points(list_of_locations[i][1], list_of_locations[i][2], user, gps_data_array, \
                                             time_start=another_day_start, time_end=day+timedelta(hours=random.randint(7,9)))
        with open(FILE_NAME, "a") as file: 
            file.write(f'It is weekday and the user stayed at from {another_day_start} till {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')

        moving_activity_time=get_moving_points(graph_proj, list_of_locations[i][0], list_of_locations[i+1][0],\
                        [list_of_locations[i][1], list_of_locations[i][2]],[list_of_locations[i+1][1], list_of_locations[i+1][2]],\
                        user, gps_data_array, time_start=stay_activity_time)
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user went to work and came there {moving_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')

        stay_activity_time=get_static_points(list_of_locations[i+1][1], list_of_locations[i+1][2], user, gps_data_array, \
                                             time_start=moving_activity_time, time_end=day+timedelta(hours=random.randint(17,19)))
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user stayed at work till {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
        
        i+=1
    else: 
        stay_activity_time=get_static_points(list_of_locations[i][1], list_of_locations[i][2], user, gps_data_array, \
                                             time_start=another_day_start, time_end=day+timedelta(hours=random.randint(22,26)))
        with open(FILE_NAME, "a") as file:
            file.write(f'It is weekend and the user decided to stay at home all day starting from {another_day_start} and "went to bed" at {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
            file.write('-'*8+"\n\n")

        
        next_time=stay_activity_time
        return stay_activity_time
    

    while i < (len(list_of_locations)-1):
        moving_activity_time=get_moving_points(graph_proj, list_of_locations[i][0], list_of_locations[i+1][0],\
                            [list_of_locations[i][1], list_of_locations[i][2]],[list_of_locations[i+1][1], list_of_locations[i+1][2]],\
                             user, gps_data_array, time_start=stay_activity_time)
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user decided to go to an event {i} and came there {moving_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
        
        stay_activity_time=get_static_points(list_of_locations[i+1][1], list_of_locations[i+1][2], user, gps_data_array,\
                                             time_start=moving_activity_time, time_end=moving_activity_time+timedelta(hours=random.randint(1,3)))
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user stayed at event {i} till {stay_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')

        i+=1
        
    if i >0:     
        moving_activity_time=get_moving_points(graph_proj, list_of_locations[i][0], list_of_locations[0][0],\
                            [list_of_locations[i][1], list_of_locations[i][2]], [list_of_locations[0][1], list_of_locations[0][2]],\
                            user, gps_data_array, time_start=stay_activity_time)
        with open(FILE_NAME, "a") as file:
            file.write(f'At {day_string} the user came to home at {moving_activity_time}\n')
            file.write(f'Current amount of data: {len(gps_data_array)}\n')
            file.write('-'*8+"\n\n")
        
        next_time=moving_activity_time
        return next_time

        

In [27]:
number_of_users=3
date_beginning = '2022-07-22'
date_end = '2022-07-26'
date_range = pd.date_range(date_beginning, date_end, freq = 'd')

In [28]:
user_locations={}
for user in range(number_of_users): 
    meaningful_locations=get_meaningful_locations(graph_proj, gdf_hw, gdf_event)
    user_locations[user]=[meaningful_locations[0], meaningful_locations[1], meaningful_locations[2]]

In [29]:
%%time
FILE_NAME=r"GPS PLOTS"
gps_data_array=[]
for user in range(number_of_users):
    home_x=gdf_hw.iloc[user_locations[user][0]]['center_x']
    home_y=gdf_hw.iloc[user_locations[user][0]]['center_y']
    home_node_id=gdf_hw.iloc[user_locations[user][0]]['nearest_node']
    work_x=gdf_hw.iloc[user_locations[user][1]]['center_x']
    work_y=gdf_hw.iloc[user_locations[user][1]]['center_y']
    work_node_id=gdf_hw.iloc[user_locations[user][1]]['nearest_node']
    
    home_list=[home_node_id, home_x, home_y]
    work_list=[work_node_id, work_x, work_y]
    time_start=date_range[0]
    with open(FILE_NAME, "a") as file: 
        file.write(f'PLOT FOR USER {user+1}\n\n')
        file.write(f'HOME_ID: {user_locations[user][0]}\n')
        file.write(f'WORK_ID: {user_locations[user][1]}\n')
        file.write(f'REGULAR_IDs: {user_locations[user][2]}\n\n')
    for i in range(len(date_range)):
        another_day_start=time_start
        day=date_range[i]
        day_string=date_range[i].strftime('%Y-%m-%d %H:%M:%S')[:10]
        day_of_week=day.isoweekday()
        if day_of_week<6: 
            number_of_events=np.random.choice([0,1,2,3], 1, p=[0.6, 0.25, 0.1, 0.05])[0]
            list_of_locations=[home_list, work_list]
        else: 
            number_of_events=np.random.choice([0,1,2,3,4], 1, p=[0.1, 0.2, 0.30, 0.25, 0.15])[0]
            list_of_locations=[home_list]
        time_start=random_plot(day_string, list_of_locations, day_of_week, number_of_events)

CPU times: total: 33.1 s
Wall time: 34.6 s


In [36]:
gps_data = pd.DataFrame(gps_data_array, columns = [ 'user', 'timestamp', 'lon', 'lat']).sort_values(by=['user','timestamp'])
print(f'Final amount of record: {len(gps_data)}')

user=2

df_1=gps_data[gps_data['user']==user][['timestamp','lon','lat']]

df_1['timestamp']=df_1['timestamp'].astype(str)

df_1['timestamp']=df_1['timestamp'].apply(lambda x: x[:10])

df_1['true_date']=pd.to_datetime(df_1['timestamp'], errors='coerce', format="%Y-%m-%d")

df_1=df_1.sort_values(by='true_date')

lat_long_list = []
date_strings=[]
for i in df_1['timestamp'].unique():
    date_strings.append(i)
    temp=[]
    for index, instance in df_1[df_1['timestamp'] == i].iterrows():
        temp.append([instance['lat'],instance['lon']])
    lat_long_list.append(temp)

m=Map(location=[58.378025, 26.728493], zoom_start=12)
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Heatmap of gps data for one user with Timestamps</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))  
HeatMapWithTime(lat_long_list,auto_play=True,index=date_strings,max_opacity=0.5).add_to(m)

home_lon, home_lat=transformer_to_WGS.transform(gdf_hw.loc[user_locations[user-1][0]]['center_x'], gdf_hw.loc[user_locations[user-1][0]]['center_y'])
work_lon, work_lat=transformer_to_WGS.transform(gdf_hw.loc[user_locations[user-1][1]]['center_x'], gdf_hw.loc[user_locations[user-1][1]]['center_y'])

event_list=[225, 1388, 56]

for event_id in event_list:
    event_lon, event_lat=transformer_to_WGS.transform(gdf_event.loc[event_id]['center_x'], gdf_event.loc[event_id]['center_y'])
    if event_id in user_locations[user-1][2]: 
        popup_text='regular_location:'+str(event_id)
    else: 
        popup_text='random_location:'+str(event_id)
    folium.Marker(
      location=[event_lat, event_lon],
      popup=popup_text
    ).add_to(m)
    
folium.Marker(
  location=[home_lat, home_lon],
  popup='home'
).add_to(m)
folium.Marker(
  location=[work_lat, work_lon],
  popup='work'
).add_to(m)
m

Final amount of record: 34827


In [31]:
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
    R = 6371 # Radius of the earth in km
    dLat = radians(lat2-lat1)
    dLon = radians(lon2-lon1)
    rLat1 = radians(lat1)
    rLat2 = radians(lat2)
    a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c # Distance in km
    return d

def calc_velocity(dist_km, time_start, time_end):
    """Return 0 if time_start == time_end, avoid dividing by 0"""
    return dist_km / ((time_end - time_start).total_seconds()/3600) if time_end > time_start else 0


def get_distance_and_velocity(df, user, lon, lat, timestamp): 
    df[timestamp]=pd.to_datetime(df[timestamp], format='%Y-%m-%d %H:%M:%S')
    df['lon_end'] = df[lon].shift(-1)
    df['lat_end'] = df[lat].shift(-1)
    df['timestamp_end']= df[timestamp].shift(-1)
    df_final=pd.DataFrame()
    for i in df[user].unique():
        df_transitional=pd.DataFrame()
        df_transitional=df[df[user]==i]
        df_final=pd.concat([df_final, df_transitional])
        df_final.drop(index=df_final.index[-1],axis=0,inplace=True)
        
    df_final['dist_km'] = df_final.apply(
        lambda row: getDistanceFromLatLonInKm(
        lat1=row[lat],
        lon1=row[lon],
        lat2=row['lat_end'],
        lon2=row['lon_end']
    ),
    axis=1)
    
    df_final['velocity_kmh'] = df_final.apply(
    lambda row: calc_velocity(
        dist_km=row['dist_km'],
        time_start=row[timestamp],
        time_end=row['timestamp_end']
    ),
    axis=1
)
    return df_final

In [37]:
df_final=get_distance_and_velocity(gps_data, 'user', 'lon', 'lat', 'timestamp')

print(df_final['velocity_kmh'].min())
print(df_final['velocity_kmh'].mean())
print(df_final['velocity_kmh'].max())

0.0
3.0789147171413016
5.912639740354129


In [38]:
df_final.iloc[220:235]

,user,timestamp,lon,lat,lon_end,lat_end,timestamp_end,dist_km,velocity_kmh
220,1,2022-07-22 08:07:23,26.696242,58.353365,26.696294,58.353376,2022-07-22 08:07:26,0.003260,3.911597
221,1,2022-07-22 08:07:26,26.696294,58.353376,26.696413,58.353373,2022-07-22 08:07:32,0.006925,4.154709
222,1,2022-07-22 08:07:32,26.696413,58.353373,26.696557,58.353387,2022-07-22 08:07:40,0.008562,3.852688
223,1,2022-07-22 08:07:40,26.696557,58.353387,26.696583,58.353382,2022-07-22 08:07:42,0.001659,2.986477
224,1,2022-07-22 08:07:42,26.696583,58.353382,26.696640,58.353391,2022-07-22 08:07:45,0.003461,4.153501
225,1,2022-07-22 08:07:45,26.696640,58.353391,26.696754,58.353389,2022-07-22 08:07:51,0.006664,3.998501
226,1,2022-07-22 08:07:51,26.696754,58.353389,26.696862,58.353397,2022-07-22 08:07:57,0.006373,3.824096
227,1,2022-07-22 08:07:57,26.696862,58.353397,26.696924,58.353395,2022-07-22 08:08:01,0.003660,3.294061
228,1,2022-07-22 08:08:01,26.696924,58.353395,26.697084,58.353387,2022-07-22 08:08:09,0.009364,4.213922
229,1,2022-07-22 08:08:09,26.697084,58.353387,26.697095,58.353400,2022-07-22 08:08:11,0.001614,2.905266


In [34]:
df_final[df_final['velocity_kmh']>5]

,user,timestamp,lon,lat,lon_end,lat_end,timestamp_end,dist_km,velocity_kmh
264,1,2022-07-22 08:10:58,26.699990,58.353543,26.700036,58.353550,2022-07-22 08:11:00,0.002789,5.019504
305,1,2022-07-22 08:14:19,26.703533,58.353580,26.703572,58.353563,2022-07-22 08:14:21,0.002998,5.395934
315,1,2022-07-22 08:15:05,26.704377,58.353544,26.704427,58.353547,2022-07-22 08:15:07,0.002943,5.296884
398,1,2022-07-22 08:21:41,26.709815,58.355530,26.709835,58.355554,2022-07-22 08:21:43,0.002981,5.366310
422,1,2022-07-22 08:23:36,26.711318,58.356263,26.711377,58.356286,2022-07-22 08:23:39,0.004294,5.152609
...,...,...,...,...,...,...,...,...,...
34647,3,2022-07-26 18:21:35,26.713486,58.372493,26.713477,58.372531,2022-07-26 18:21:38,0.004201,5.040959
34722,3,2022-07-26 18:27:47,26.711271,58.375661,26.711228,58.375675,2022-07-26 18:27:49,0.002968,5.343194
34730,3,2022-07-26 18:28:27,26.711048,58.376002,26.711014,58.376023,2022-07-26 18:28:29,0.003017,5.429824
34791,3,2022-07-26 18:33:29,26.709398,58.378583,26.709355,58.378596,2022-07-26 18:33:31,0.002856,5.140134
